<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Выводы:" data-toc-modified-id="Выводы:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы:</a></span></li>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
print(data['Tenure'].isna().sum() / len(data) * 100, '% пропусков в Tenure')

9.09 % пропусков в Tenure


In [5]:
data['Tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

In [6]:
data = data.dropna()
data['Tenure'] = data['Tenure'].astype('int')

Так как пропусков всего 9%, я решила их удалить

In [7]:
data_ohe = pd.get_dummies(data.drop(['Surname'], axis=1), drop_first=True)

In [8]:
data_ohe.head()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,1,15634602,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,2,15647311,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0
2,3,15619304,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,4,15701354,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,5,15737888,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0


In [9]:
target = data_ohe['Exited']
features = data_ohe.drop(['CustomerId','RowNumber', 'Exited'], axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
features, target, test_size=0.4, random_state=12345, stratify=target)

Удаляю 'Surname', так как этот столбец ни на что не влияет, также как и ID. Столбец 'RowNumber' тоже не нужен, так как он отображает номер строки.

In [10]:
features_valid, features_test, target_valid, target_test = train_test_split(
features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)

## Исследование задачи

In [11]:
print(data.query('Exited == 0')['Exited'].count())
print(data.query('Exited == 1')['Exited'].count())

7237
1854


Прослеживается явный дисбаланс

In [12]:
features_train.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
9723,526,32,7,125540.05,1,0,0,86786.41,0,0,1
1224,500,47,8,128486.11,1,1,0,179227.12,0,1,0
8377,802,40,4,0.00,2,1,1,81908.09,0,1,1
8014,731,39,2,126816.18,1,1,1,74850.93,0,1,0
2491,612,26,4,0.00,2,1,1,179780.74,0,1,0


In [13]:
scaler = StandardScaler()
scaler.fit(features_train)
features_train_scaled = scaler.transform(features_train)
features_valid_scaled = scaler.transform(features_valid)
features_test_scaled = scaler.transform(features_test)

In [14]:
model = LogisticRegression()
model.fit(features_train_scaled, target_train)
predictions_valid = model.predict(features_valid_scaled)

print(f1_score(target_valid, predictions_valid))
print()
print(confusion_matrix(target_valid, predictions_valid))

0.3306772908366534

[[1399   48]
 [ 288   83]]


In [15]:
for depth in range(1, 20):
    model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=depth)
    model.fit(features_train_scaled, target_train)
    predicted_valid = model.predict(features_valid_scaled)
    print(f1_score(target_valid, predicted_valid))

0.0
0.16748768472906403
0.23584905660377362
0.4273858921161826
0.5057471264367815
0.5306122448979592
0.5525846702317291
0.5729166666666666
0.5956006768189509
0.5763293310463121
0.5898305084745763
0.6076794657762938
0.5976627712854758
0.6075533661740558
0.6019736842105263
0.5953947368421053
0.5894039735099337
0.5974025974025975
0.6121112929623569


In [16]:
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train_scaled, target_train)
    predicted_valid = model.predict(features_valid_scaled)
    print(f1_score(target_valid, predicted_valid))

0.0
0.5240464344941957
0.5208681135225375
0.5127272727272727
0.5733788395904437
0.5797598627787307
0.5815126050420169
0.5463576158940396
0.5382059800664452
0.5486443381180223
0.5214814814814815
0.5007541478129715
0.5021770682148041
0.4921316165951359
0.49108367626886146
0.4918918918918919
0.5040431266846361
0.4823989569752281
0.48339973439575035


*** Вывод:***

Наилучшей моделью является RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=19), f1 = 0.612

## Борьба с дисбалансом

In [18]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [19]:
features_upsampled, target_upsampled = upsample(
    pd.DataFrame(features_train_scaled, index=features_train.index), target_train, 4)

In [20]:
model = LogisticRegression(random_state=12345, solver='liblinear', class_weight='balanced')
model.fit(features_train_scaled, target_train)
predicted_valid = model.predict(features_valid_scaled)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5079365079365079


In [21]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled, target_upsampled)
predicted_valid = model.predict(features_valid_scaled)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5050878815911193


Метрика f1 незначительно уменьшилась.

In [22]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [23]:
features_downsampled, target_downsampled = downsample(
    pd.DataFrame(features_train_scaled, index=features_train.index), target_train, 0.26)

model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_downsampled, target_downsampled)
predicted_valid = model.predict(features_valid_scaled)
print("F1:", f1_score(target_valid, predicted_valid))

F1: 0.5163704396632366


Уменьшение выборки немного увеличивает метрику f1, но данный метод не совсем нам подходит, потому что мы можем потерять важные данные.

In [24]:
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_train_scaled, target_train)

probabilities_valid = model.predict_proba(features_valid_scaled)
probabilities_one_valid = probabilities_valid[:, 1]

roc_auc = roc_auc_score(target_valid, probabilities_one_valid)
roc_auc

0.7893047610354726

In [25]:
for depth in range(1, 20):
    model = RandomForestClassifier(random_state=12345, n_estimators=20, max_depth=depth)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid_scaled)
    print(f1_score(target_valid, predicted_valid))

0.5433746425166825
0.5515210991167812
0.5737704918032787
0.6025641025641026
0.6081229418221735
0.6072607260726073
0.6228070175438596
0.6341463414634148
0.617169373549884
0.6223277909738717
0.6287128712871287
0.646074646074646
0.6216216216216216
0.6198347107438017
0.6253521126760563
0.6326241134751773
0.5977011494252873
0.5875370919881306
0.6142649199417759


In [26]:
for depth in range(1, 20):
    model = RandomForestClassifier(random_state=12345, n_estimators=10, max_depth=depth)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid_scaled)
    print(f1_score(target_valid, predicted_valid))

0.4638157894736842
0.5780104712041885
0.5516569200779727
0.6077348066298344
0.6051743532058492
0.6129753914988815
0.6187845303867403
0.6285714285714286
0.6093928980526919
0.6251497005988025
0.6140776699029126
0.5989583333333334
0.6143617021276596
0.606060606060606
0.5847457627118643
0.6005665722379602
0.5868613138686132
0.5847076461769115
0.593245227606461


In [27]:
for depth in range(1, 20):
    model = RandomForestClassifier(random_state=12345, n_estimators=80, max_depth=depth)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid_scaled)
    print(f1_score(target_valid, predicted_valid))

0.5387596899224806
0.5478087649402391
0.5801047120418849
0.5908607863974495
0.6080347448425624
0.6157253599114065
0.6298342541436462
0.6261467889908257
0.6313364055299538
0.6278240190249703
0.631055900621118
0.6369593709043251
0.6327077747989276
0.6358543417366946
0.6304044630404463
0.6293103448275863
0.6330935251798562
0.6193921852387844
0.6226138032305433


Модель леса дает результат 0.646 при n_estimators=20 и max_depth=12

In [28]:
for depth in range(1, 20):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_upsampled, target_upsampled)
    predicted_valid = model.predict(features_valid_scaled)
    print(f1_score(target_valid, predicted_valid))

0.5092402464065708
0.5376128385155466
0.5376128385155466
0.5522682445759368
0.5711481844946025
0.5825049701789264
0.5628042843232717
0.5813449023861171
0.5548245614035088
0.531284302963776
0.5378346915017462
0.5087924970691676
0.500590318772137
0.5115712545676004
0.5155666251556662
0.5114503816793893
0.523936170212766
0.49736842105263157
0.5190039318479686


При max_depth = 6 дерево дает нам наилучшее решение: 0.5825

Вывод: Здесь я боролась с дисбалансом техникой upsample, она лучше downsample, так как мы не теряем часть данных.

Модель леса дает лучший результат 0.646 при n_estimators=20 и max_depth=12, тогда как дерево дает нам наилучшее решение: 0.5825. Модель логистической регресии дает результат еще хуже: F1 = 0.5

## Тестирование модели

In [29]:
model = RandomForestClassifier(n_estimators=20, max_depth=12, random_state=12345)
model.fit(features_upsampled, target_upsampled)
predicted_test = model.predict(features_test_scaled)
f1_score(target_test, predicted_test)

0.5923984272608126

## Выводы:



Передо мной стояла задача спрогнозировать, уйдёт клиент из банка в ближайшее время или нет.

- На несбалансированных данных я получила следующий результат: наилучшей моделью является RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=19), f1 = 0.612

- Здесь я боролась с дисбалансом техникой upsample, она лучше downsample, так как мы не теряем часть данных.

- На сбалансированных данных модель леса дает лучший результат 0.646 при n_estimators=20 и max_depth=12, тогда как дерево дает нам наилучшее решение: 0.5825. Модель логистической регресии дает результат еще хуже: f1 = 0.5


При тестировании модели я получила метрику f1 равную 0.592.